<a href="https://colab.research.google.com/github/60hr00t/draft1/blob/main/Vet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece fastapi uvicorn pyngrok nest_asyncio streamlit requests gTTS fuzzywuzzy python-Levenshtein streamlit-image-select streamlit-calendar pandas streamlit-mic-recorder openai-whisper

In [3]:
# Detecting CUDA GPU with PyTorch
import torch, platform

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [4]:
# ==========================================================
# Upload a CSV with “prompt” and “response” columns in Colab
# and set `CSV_PATH` to the selected file.
# ==========================================================

import io, pandas as pd
from google.colab import files

print("Upload a CSV with columns: prompt,response")
uploaded = files.upload()
if uploaded:
    fname = list(uploaded.keys())[0]
    CSV_PATH = f"/content/{fname}"
    print("Using uploaded file:", CSV_PATH)


Upload a CSV with columns: prompt,response


Saving Vet.csv - Vet.csv to Vet.csv - Vet.csv
Using uploaded file: /content/Vet.csv - Vet.csv


In [5]:
# ==================================================
# Fuzzy match answers from CSV and render as plain
# text or option buttons
# ==================================================
from fuzzywuzzy import fuzz
df = pd.read_csv(CSV_PATH)
required = {"prompt", "response"}
assert required.issubset(df.columns), f"CSV must include: {required}. Found: {df.columns.tolist()}"
print(f"Loaded {len(df)} rows.")

# This function returns the best-matching response string for user_msg
# or None if the best score is below threshold.
def answer_from_csv(user_msg, df, threshold=90):
    best_score = 0
    best_response = None
    for _, row in df.iterrows():
        score = fuzz.partial_ratio(user_msg.lower(), row["prompt"].lower())
        if score > best_score:
            best_score = score
            best_response = row["response"]
    return best_response if best_score >= threshold else None
     # Check if response contains semicolons
    if ";" in response:
        options = [opt.strip() for opt in response.split(";")]
        st.markdown(f"**{prompt.capitalize()} Options:**")
    for i, option in enumerate(options):
            if st.button(option, key=f"{prompt}_{i}"):
                st.chat_message("user").markdown(option)
                reply = get_bot_response(option)  # Send to your backend
                st.chat_message("assistant").markdown(reply)
    else:
# If no semicolon, show as regular response
        st.markdown(f"**{prompt.capitalize()}:** {response}")




Loaded 2 rows.


In [6]:
# =====================================================================
# This snippet downloads and initializes Google’s FLAN-T5 (small) model
# and its tokenizer using the  Transformers library, then puts the model
# in evaluation mode for inference.
# =====================================================================
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [7]:
# ==========================================================================
# This section allows the FastAPI app exposes a chat endpoint that first looks
# up a reply in a CSV via answer_from_csv, falls back to a generative model
# if no strong match exists, synthesizes the reply to MP3 with gTTS, and serves
# the audio from a static /audio path.
# ==========================================================================
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
import uvicorn, nest_asyncio, threading, time, uuid, os
from pyngrok import conf, ngrok
from gtts import gTTS

app = FastAPI()
app.mount("/audio", StaticFiles(directory="/content", html=True), name="audio")

class Prompt(BaseModel):
    prompt: str

@app.post("/respond")
async def respond(request: Prompt):
    user_msg = request.prompt.strip()
    csv_answer = answer_from_csv(user_msg, df, threshold=90)
    filename = f"audio_{uuid.uuid4().hex[:8]}.mp3"
    filepath = f"/content/{filename}"

    if csv_answer:
        gTTS(text=csv_answer, lang='en').save(filepath)
        return {"response": csv_answer, "audio_path": f"/audio/{filename}"}

    prompt_text = f"You are a helpful cultural assistant. Answer clearly.\nUser: {user_msg}\nAssistant:"
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=120)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    gTTS(text=reply, lang='en').save(filepath)
    return {"response": reply, "audio_path": f"/audio/{filename}"}

@app.get("/audio/{filename}")
async def get_audio(filename: str):
    return FileResponse(path=f"/content/{filename}", media_type="audio/mpeg")


In [8]:
# =====================================================================
# This snippet runs the FastAPI app with Uvicorn in a background thread
# (using nest_asyncio), opens an ngrok tunnel with the auth token, then
# builds and prints the public /respond endpoint (FASTAPI_URL) and the
# base URL for serving audio (AUDIO_BASE_URL).
# ======================================================================
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

nest_asyncio.apply()
threading.Thread(target=run_api, daemon=True).start()
time.sleep(5)

conf.get_default().auth_token = "36Jyi3nI1rdAxQw3UOMT4SrCerI_3DHRyYHWaKb9PsJorLM3n"
public_url = ngrok.connect(8000)
FASTAPI_URL = f"{public_url.public_url}/respond"
AUDIO_BASE_URL = public_url.public_url
print("FASTAPI URL:", FASTAPI_URL)
print("AUDIO BASE:", AUDIO_BASE_URL)

INFO:     Started server process [353]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FASTAPI URL: https://unvibrated-tubercularly-darwin.ngrok-free.dev/respond
AUDIO BASE: https://unvibrated-tubercularly-darwin.ngrok-free.dev


In [9]:
import os
os.environ["SMTP_HOST"]="smtp.gmail.com"
os.environ["SMTP_PORT"]="587"
os.environ["SMTP_USER"]=""
os.environ["SMTP_PASS"]="your_app_password"
os.environ["FROM_EMAIL"]=""

In [10]:
%%writefile vet_finder_app.py
# vet_finder_app.py - BLUE & YELLOW VERSION WITH 100 BASIC RESPONSES
import streamlit as st
import requests
import time
import html
import tempfile
import pandas as pd
import os
from typing import Tuple, Optional
from datetime import datetime

# Optional components
try:
    from streamlit_image_select import image_select
except:
    image_select = None

try:
    from streamlit_mic_recorder import mic_recorder
    import whisper
    WHISPER_OK = True
except:
    WHISPER_OK = False

# Import API URLs
try:
    from config import FASTAPI_URL, AUDIO_BASE_URL
except:
    FASTAPI_URL = ""
    AUDIO_BASE_URL = ""

# --- Imgur assets ---
USER_AVATARS = [
    "https://i.imgur.com/z0TxKqZ.png",
    "https://i.imgur.com/olgbtrK.png",
    "https://i.imgur.com/86azvSc.png",
    "https://i.imgur.com/exSI77K.png",
]
BOT_AVATAR = "https://i.imgur.com/PCDvPM0.png"

# --- Page config ---
st.set_page_config(
    page_title="Jamaica Vet Finder",
    page_icon=BOT_AVATAR,
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Load CSV function ---
@st.cache_data
def load_csv_data():
    """Load the CSV file with prompt/response pairs"""
    csv_path = "/content/Vet.csv - Vet.csv"
    if os.path.exists(csv_path):
        try:
            df = pd.read_csv(csv_path)
            if "prompt" in df.columns and "response" in df.columns:
                return df
        except Exception as e:
            st.error(f"Error loading CSV: {e}")
    return pd.DataFrame(columns=["prompt", "response"])

# Load CSV data
df = load_csv_data()

# --- 100 Basic Questions/Commands with Responses ---
BASIC_COMMANDS = {
    # Greetings
    "hi": "Hi there! 👋 I'm Dr. Vetman. How can I help you with your pet today?",
    "hello": "Hello! 🐾 Welcome to Jamaica Vet Finder. What can I assist you with?",
    "hey": "Hey! 😊 How can I help you find veterinary care in Jamaica?",
    "good morning": "Good morning! ☀️ Hope you and your pet are doing well today! How can I help?",
    "good afternoon": "Good afternoon! 🌤️ What can I do for you and your furry friend today?",
    "good evening": "Good evening! 🌙 How can I assist you with your pet care needs?",
    "good night": "Good night! 🌟 Sweet dreams to you and your pet!",
    "greetings": "Greetings! Welcome to Jamaica Vet Finder. How may I assist you today?",
    "howdy": "Howdy! 🤠 Ready to help you find the best vet care in Jamaica!",
    "yo": "Yo! What's up? How can I help you and your pet today?",

    # Farewells
    "bye": "FEEDBACK_TRIGGER",
    "goodbye": "FEEDBACK_TRIGGER",
    "see you": "FEEDBACK_TRIGGER",
    "see ya": "FEEDBACK_TRIGGER",
    "later": "See you later! 👋 Take care of your pet!",
    "catch you later": "Catch you later! Stay safe with your furry friend! 🐾",
    "farewell": "FEEDBACK_TRIGGER",
    "peace": "Peace out! ✌️ Take care of your pet!",
    "gotta go": "No problem! Come back anytime you need help! 👋",
    "take care": "You too! Take care of yourself and your pet! 💙",

    # Thanks
    "thanks": "You're welcome! 💙 Happy to help anytime!",
    "thank you": "My pleasure! 🌟 Feel free to ask if you need anything else.",
    "thank you so much": "You're very welcome! Always happy to help! 😊",
    "thanks a lot": "No problem at all! That's what I'm here for! 🐾",
    "many thanks": "You're most welcome! Glad I could help! 💛",
    "appreciate it": "I appreciate you too! Happy to assist! 😊",
    "thx": "No problem! Anytime! 👍",
    "ty": "You're welcome! 😊",

    # About Dr. Vetman
    "how are you": "I'm doing great, thanks for asking! 😊 Ready to help you find the best vet care in Jamaica. How about you and your pet?",
    "what are you": "I'm Dr. Vetman, your virtual veterinary assistant! 🩺 I help pet owners across Jamaica find vet clinics, emergency care, and pet-care tips. 🐾",
    "who are you": "I'm Dr. Vetman! 🐕‍⚕️ I'm here to help you locate veterinary services and provide pet-care guidance across Jamaica.",
    "what is your name": "I'm Dr. Vetman, your friendly virtual vet assistant! Nice to meet you! 😊",
    "who made you": "I was created to help pet owners across Jamaica find veterinary care and get pet health advice! 🐾",
    "what do you do": "I help you find vet clinics, emergency care, vaccination info, pet care tips, and answer questions about pet health in Jamaica! 🩺",
    "are you a real vet": "I'm a virtual assistant, but I provide information based on veterinary expertise to help you care for your pet! 🐾",
    "are you ai": "Yes, I'm an AI assistant designed to help you find veterinary services and pet care information! 💙",
    "how old are you": "I'm timeless! I'm here whenever you need help with your pet! 😊",

    # Help & Capabilities
    "help": "I can help you with:\n🏥 Finding vet clinics near you\n🚨 Emergency vet services\n💊 Pet care tips & advice\n💉 Vaccination information\n🐾 And much more! Just ask me anything. 😊",
    "what can you do": "I can help you find vets in Jamaica, provide emergency contacts, answer pet care questions, give vaccination info, and much more! 🐾",
    "help me": "Of course! What do you need help with? Vet clinics, emergency care, or pet health advice? 😊",
    "i need help": "I'm here for you! What's going on with your pet? 🐾",
    "can you help me": "Absolutely! That's what I'm here for. What do you need? 💙",
    "what services": "I provide vet clinic locations, emergency contacts, pet care tips, vaccination schedules, and general pet health guidance! 🩺",
    "how can you help": "I can locate vets near you, provide emergency numbers, answer pet health questions, and give care advice! 😊",
    "assist me": "Happy to assist! What can I help you with today? 🐾",

    # Pleasantries
    "how's it going": "Going great! How can I help you and your pet today? 😊",
    "what's up": "Not much! Just ready to help you with your pet care needs. What's up with you? 🐾",
    "how are things": "Things are wonderful! How can I assist you today? 💙",
    "nice to meet you": "Nice to meet you too! 🤝 Excited to help you take care of your pet!",
    "pleasure to meet you": "The pleasure is mine! How can I help your pet today? 😊",
    "you're welcome": "😊 Always happy to help! Is there anything else you'd like to know?",
    "no problem": "😊 Is there anything else I can help you with?",
    "all good": "Great! Let me know if you need anything else! 👍",

    # General Questions
    "what time is it": "I don't have access to the current time, but I'm available 24/7 to help you with pet care! 🕐",
    "what day is it": "I'm here every day to help you! What can I do for your pet today? 📅",
    "where are you": "I'm right here in the Jamaica Vet Finder app, ready to help! 💙",
    "are you busy": "Never too busy for you! What do you need help with? 😊",
    "are you there": "Yes, I'm here! How can I help you today? 🐾",
    "can you hear me": "I can read your messages! What would you like to know? 😊",
    "are you real": "I'm a virtual assistant, but I'm really here to help you! 💙",
    "do you sleep": "Nope! I'm available 24/7 for all your pet care needs! 🌟",
    "are you human": "I'm an AI assistant, but I'm here to help you just like a human would! 😊",

    # Emotions & Reactions
    "awesome": "Glad you think so! 😊 How else can I help?",
    "cool": "Thanks! What else can I do for you? 😎",
    "great": "Wonderful! Is there anything else you need? 💙",
    "nice": "Thanks! Let me know if you need anything else! 😊",
    "wow": "😊 How can I help you today?",
    "amazing": "Thank you! How can I assist you further? 🌟",
    "perfect": "Glad it worked out! Anything else I can help with? ✨",
    "excellent": "Thank you! What else do you need? 😊",
    "fantastic": "Great! How else can I help? 🎉",
    "wonderful": "Thank you! What can I do for you? 💛",

    # Affirmations
    "yes": "Great! How can I help you? 😊",
    "yeah": "Awesome! What do you need? 🐾",
    "yep": "Perfect! What can I do for you? 👍",
    "sure": "Great! How can I assist? 💙",
    "okay": "Perfect! What's next? 😊",
    "ok": "Alright! How can I help? 👍",
    "no": "No problem! Let me know if you change your mind or need something else! 😊",
    "nope": "That's okay! Is there something else I can help with? 🐾",
    "maybe": "Take your time! I'm here when you're ready. 😊",
    "perhaps": "Let me know if you need any help deciding! 💙",

    # Pet-Related Questions
    "i have a dog": "Wonderful! Dogs are amazing companions. How can I help with your dog's care? 🐕",
    "i have a cat": "Cats are great! How can I assist with your cat's health? 🐈",
    "i love pets": "Me too! Pets are wonderful. How can I help you care for yours? 🐾",
    "pets are great": "Absolutely! They bring so much joy. What can I do for your pet? 💙",
    "do you like animals": "I love helping pet owners care for their animals! 🐾 How can I help yours?",
    "what about birds": "I can help with bird care too! What do you need to know? 🦜",
    "what about rabbits": "Happy to help with rabbit care! What's your question? 🐰",

    # Compliments to Bot
    "you're helpful": "Thank you! 😊 That's what I'm here for! How else can I help?",
    "you're smart": "Thanks! I try my best to help you! 💙 What else do you need?",
    "you're good": "Thank you! I appreciate that! 😊 What can I do for you?",
    "you're the best": "Aww, thanks! You're pretty great too! 💛 How can I help?",
    "i like you": "I like you too! 😊 How can I assist you today?",
    "you're awesome": "Thank you! You're awesome too! 🌟 What do you need?",
    "good job": "Thanks! Happy to help! 😊 What's next?",
    "well done": "Thank you! 💙 How else can I assist?",

    # Questions About Jamaica
    "where in jamaica": "I can help you find vets all across Jamaica! Which parish are you in? 🇯🇲",
    "jamaica": "Yes! I specialize in helping pet owners across all of Jamaica! 🇯🇲 Which area do you need?",
    "kingston": "Looking for vets in Kingston? I can help! What specifically do you need? 🏥",
    "montego bay": "Need a vet in Montego Bay? I can help you find one! 🐾",
    "spanish town": "Looking for vets in Spanish Town? Let me help you! 🩺",

    # Random Questions
    "tell me a joke": "Why did the dog go to the vet? Because he was feeling ruff! 😄 How can I help you today?",
    "are you funny": "I try! 😊 But I'm better at helping you find vet care! What do you need?",
    "sing a song": "I'm better at finding vets than singing! 🎵 How can I help your pet today?",
    "tell me a story": "Here's a story: Once upon a time, a pet owner found the perfect vet through Dr. Vetman! Now, how can I help your story? 😊",
    "what's your favorite color": "Blue and yellow, like Jamaica's beautiful beaches and sunshine! ☀️ Now, how can I help you?",
}

def check_basic_command(msg: str) -> Optional[str]:
    """Check if message matches a basic command"""
    msg_lower = msg.lower().strip()
    # Remove common punctuation
    msg_lower = msg_lower.replace('?', '').replace('!', '').replace('.', '').replace(',', '')
    return BASIC_COMMANDS.get(msg_lower)

# --- Fuzzy matching function ---
from fuzzywuzzy import fuzz

def answer_from_csv(user_msg, df, threshold=90):
    """Find best matching response from CSV"""
    best_score = 0
    best_response = None
    for _, row in df.iterrows():
        score = fuzz.partial_ratio(user_msg.lower(), row["prompt"].lower())
        if score > best_score:
            best_score = score
            best_response = row["response"]
    return best_response if best_score >= threshold else None

# --- Blue & Yellow CSS ---
st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');

    * {
        font-family: 'Inter', sans-serif;
    }

    :root {
        --blue-primary: #0066cc;
        --blue-light: #3399ff;
        --blue-lighter: #e6f2ff;
        --blue-dark: #004d99;
        --yellow-primary: #ffcc00;
        --yellow-light: #ffe666;
        --yellow-lighter: #fff9e6;
        --yellow-dark: #cc9900;
        --text: #062f4f;
        --muted: #6b7280;
    }

    /* Blue & Yellow gradient background */
    .stApp {
        background: linear-gradient(135deg, #e6f2ff 0%, #ffffff 25%, #fff9e6 50%, #e6f2ff 75%, #ffffff 100%);
        background-attachment: fixed;
    }

    /* Modern onboarding */
    .onboard-modern {
        text-align: center;
        padding: 60px 24px;
        background: linear-gradient(135deg, #ffffff 0%, #e6f2ff 50%, #fff9e6 100%);
        border-radius: 24px;
        margin: 40px auto;
        max-width: 600px;
        box-shadow: 0 8px 32px rgba(0, 102, 204, 0.15);
        border: 3px solid var(--yellow-primary);
    }

    .onboard-modern h2 {
        background: linear-gradient(135deg, var(--blue-primary) 0%, var(--yellow-dark) 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        font-weight: 700;
        font-size: 2.4rem;
        margin-bottom: 12px;
    }

    .onboard-modern p {
        color: var(--muted);
        font-size: 1.1rem;
        margin-bottom: 32px;
    }

    [data-testid="collapsedControl"] { display: none !important; }

    [data-testid="stSidebar"] {
        min-width: 300px !important;
        width: 300px !important;
        background: linear-gradient(180deg, #ffffff 0%, #e6f2ff 50%, #fff9e6 100%) !important;
        backdrop-filter: blur(10px);
        border-right: 3px solid var(--yellow-primary);
    }

    .profile-pill {
        display: flex;
        align-items: center;
        gap: 12px;
        background: linear-gradient(135deg, var(--blue-lighter) 0%, var(--yellow-lighter) 100%);
        padding: 12px 14px;
        border-radius: 18px;
        box-shadow: 0 4px 16px rgba(0, 102, 204, 0.15);
        border: 2px solid var(--yellow-primary);
    }

    .profile-pill img {
        width: 52px;
        height: 52px;
        border-radius: 14px;
        object-fit: cover;
        border: 3px solid var(--blue-primary);
    }

    .chat-wrapper {
        max-width: 980px;
        margin: 24px auto 80px auto;
        padding: 0 16px;
    }

    .chat-row {
        display: flex;
        gap: 14px;
        margin: 14px 0;
        align-items: flex-start;
        animation: slideIn 0.4s ease-out;
    }

    @keyframes slideIn {
        from {
            opacity: 0;
            transform: translateY(15px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }

    .chat-row.user { justify-content: flex-end; }

    .bubble {
        max-width: 76%;
        padding: 16px 20px;
        border-radius: 18px;
        box-shadow: 0 4px 16px rgba(0, 102, 204, 0.1);
        line-height: 1.7;
        position: relative;
    }

    .bubble.user {
        background: linear-gradient(135deg, var(--blue-primary) 0%, var(--blue-light) 100%);
        color: white;
        border-bottom-right-radius: 4px;
        box-shadow: 0 4px 20px rgba(0, 102, 204, 0.3);
    }

    .bubble.bot {
        background: linear-gradient(135deg, var(--blue-lighter) 0%, var(--yellow-lighter) 100%);
        color: var(--text);
        border: 2px solid var(--yellow-primary);
        border-bottom-left-radius: 4px;
    }

    .avatar {
        width: 50px;
        height: 50px;
        border-radius: 15px;
        object-fit: cover;
        border: 3px solid var(--yellow-primary);
        box-shadow: 0 4px 12px rgba(255, 204, 0, 0.3);
    }

    .sender {
        font-weight: 600;
        font-size: 0.85rem;
        margin-bottom: 6px;
        opacity: 0.9;
    }

    .msg-text {
        font-size: 0.95rem;
        white-space: pre-wrap;
    }

    /* Input area */
    [data-testid="stTextInput"] input {
        border-radius: 14px !important;
        border: 2px solid var(--blue-light) !important;
        padding: 14px 18px !important;
        font-size: 0.95rem !important;
        background: white !important;
        box-shadow: 0 2px 8px rgba(0, 102, 204, 0.08) !important;
    }

    [data-testid="stTextInput"] input:focus {
        border-color: var(--yellow-primary) !important;
        box-shadow: 0 0 0 4px rgba(255, 204, 0, 0.2) !important;
    }

    .stButton button {
        border-radius: 14px !important;
        font-weight: 600 !important;
        transition: all 0.3s ease !important;
        box-shadow: 0 2px 8px rgba(0, 102, 204, 0.15) !important;
        background: linear-gradient(135deg, var(--blue-primary) 0%, var(--blue-light) 100%) !important;
        color: white !important;
        border: 2px solid var(--yellow-primary) !important;
    }

    .stButton button:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 20px rgba(255, 204, 0, 0.4) !important;
        background: linear-gradient(135deg, var(--blue-dark) 0%, var(--blue-primary) 100%) !important;
    }

    /* Feedback modal */
    .feedback-modal {
        background: linear-gradient(135deg, #ffffff 0%, var(--blue-lighter) 50%, var(--yellow-lighter) 100%);
        border-radius: 20px;
        padding: 32px;
        margin: 24px 0;
        border: 3px solid var(--yellow-primary);
        box-shadow: 0 8px 32px rgba(0, 102, 204, 0.2);
        animation: modalSlideIn 0.4s ease-out;
    }

    @keyframes modalSlideIn {
        from {
            opacity: 0;
            transform: scale(0.95);
        }
        to {
            opacity: 1;
            transform: scale(1);
        }
    }

    /* Badges */
    .badge {
        display: inline-block;
        padding: 6px 12px;
        border-radius: 20px;
        font-size: 0.85rem;
        font-weight: 600;
        margin: 4px;
    }

    .badge-blue {
        background: linear-gradient(135deg, var(--blue-primary) 0%, var(--blue-light) 100%);
        color: white;
    }

    .badge-yellow {
        background: linear-gradient(135deg, var(--yellow-primary) 0%, var(--yellow-light) 100%);
        color: var(--text);
    }

    #MainMenu { visibility: hidden; }
    footer { visibility: hidden; }

    /* Header */
    h1 {
        background: linear-gradient(135deg, var(--blue-primary) 0%, var(--yellow-dark) 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        font-weight: 700;
    }

    /* Stat cards */
    .stat-card {
        background: linear-gradient(135deg, #ffffff 0%, var(--blue-lighter) 100%);
        padding: 16px;
        border-radius: 16px;
        border: 2px solid var(--yellow-primary);
        margin: 8px 0;
        box-shadow: 0 4px 12px rgba(0, 102, 204, 0.1);
    }
    </style>
    """, unsafe_allow_html=True)

# --- Helper functions ---
def get_bot_response(prompt: str, timeout: int = 30) -> Tuple[str, Optional[str]]:
    """Get response from FastAPI backend"""
    if not FASTAPI_URL:
        return ("I'm currently in offline mode, but I can still help with basic questions! Try asking about vets in specific areas. 🌟", None)

    try:
        res = requests.post(FASTAPI_URL, json={"prompt": prompt}, timeout=timeout)
        if res.status_code != 200:
            return (f"Sorry, I'm having trouble connecting right now. Please try again. 😊", None)

        data = res.json()
        return (data.get("response", "No response."), data.get("audio_path"))
    except Exception as e:
        return ("I'm having connection issues. But I can still help with basic questions! 💙", None)

# Whisper STT
if WHISPER_OK:
    @st.cache_resource
    def _load_whisper_model():
        return whisper.load_model("tiny")

    def _stt_from_bytes(wav_bytes: bytes) -> str:
        try:
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                tmp.write(wav_bytes)
                tmp.flush()
                model = _load_whisper_model()
                result = model.transcribe(tmp.name, language="en")
                return (result.get("text") or "").strip()
        except Exception as e:
            st.error(f"Speech-to-text failed: {e}")
            return ""
else:
    def _stt_from_bytes(_):
        return ""

# --- Session state ---
if "user_name" not in st.session_state:
    st.session_state.user_name = ""
if "avatar" not in st.session_state:
    st.session_state.avatar = USER_AVATARS[0]
if "avatar_chosen" not in st.session_state:
    st.session_state.avatar_chosen = False
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []
if "show_feedback" not in st.session_state:
    st.session_state.show_feedback = False
if "star_rating" not in st.session_state:
    st.session_state.star_rating = 0
if "message_count" not in st.session_state:
    st.session_state.message_count = 0

# --- Onboarding: Name ---
if not st.session_state.user_name:
    st.markdown("""
        <div class='onboard-modern'>
            <h2>🐾 Welcome to Jamaica Vet Finder</h2>
            <p>Let's get started — what's your name?</p>
        </div>
    """, unsafe_allow_html=True)

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        name = st.text_input("", key="name_input", placeholder="Enter your name...", label_visibility="collapsed")

        if st.button("Continue →", key="name_submit", use_container_width=True):
            if name.strip():
                st.session_state.user_name = name.strip()
                st.rerun()
            else:
                st.warning("Please enter your name 😊")
    st.stop()

# --- Onboarding: Avatar ---
if not st.session_state.avatar_chosen:
    st.markdown("""
        <div class='onboard-modern'>
            <h2>Choose Your Avatar</h2>
            <p>Pick an avatar that represents you 🎨</p>
        </div>
    """, unsafe_allow_html=True)

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        if image_select is not None:
            sel = image_select("", USER_AVATARS, use_container_width=True, return_value="index")
            if sel is not None:
                st.session_state.avatar = USER_AVATARS[sel]
        else:
            cols = st.columns(len(USER_AVATARS))
            for i, a in enumerate(USER_AVATARS):
                with cols[i]:
                    if st.button(f"Avatar {i+1}", key=f"av_{i}"):
                        st.session_state.avatar = a

        if st.session_state.avatar:
            st.image(st.session_state.avatar, width=100)

        if st.button("Continue →", key="avatar_submit", use_container_width=True):
            if not st.session_state.avatar:
                st.warning("Please pick an avatar first 🎨")
            else:
                st.session_state.avatar_chosen = True
                st.rerun()
    st.stop()

# --- Sidebar ---
with st.sidebar:
    st.markdown(
        f"""<div class='profile-pill'>
            <img src='{BOT_AVATAR}'/>
            <div>
                <div style='font-weight:600;color:var(--blue-primary);'>Dr. Vetman</div>
                <div style='font-size:0.85rem;color:var(--muted);'>Virtual Vet Assistant 🩺</div>
            </div>
        </div>""",
        unsafe_allow_html=True
    )
    st.markdown("---")
    st.markdown(f"### 👋 Hello {st.session_state.user_name}!")
    st.write("Welcome! Use the chat to find vets, emergency care, and pet-care tips across Jamaica. 🐾")

    # Stats
    st.markdown("---")
    st.markdown("### 📊 Your Stats")
    st.markdown(f"""
        <div class='stat-card'>
            <div style='font-size:1.8rem;font-weight:700;color:var(--blue-primary);'>{st.session_state.message_count}</div>
            <div style='color:var(--muted);'>Messages Sent</div>
        </div>
    """, unsafe_allow_html=True)

    st.markdown("---")
    st.markdown("### 🩺 About Dr. Vetman")
    st.write("Dr. Vetman is your friendly virtual assistant helping pet owners across Jamaica find the best veterinary care. 💙")
    st.markdown("---")
    st.markdown("### 📞 Emergency Contacts")
    st.write("🚨 **Jamaica Vet Emergency**  \n(876) 555-0123")
    st.write("🏥 **Kingston Central Vet**  \n(876) 555-0456")
    st.markdown("---")
    st.markdown("### ⚙️ App Controls")

    if st.button("🧹 Clear Chat", key="sidebar_clear", use_container_width=True):
        st.session_state.chat_history = []
        st.success("Chat cleared! 🌟")
        st.rerun()

    if st.button("🔄 Reset App", key="sidebar_reset", use_container_width=True):
        for key in list(st.session_state.keys()):
            del st.session_state[key]
        st.success("App reset! ✨")
        st.rerun()

    st.markdown("---")
    st.markdown("### 💡 Knowledge Base")
    st.markdown(f'<span class="badge badge-blue">✓ {len(df)} Q&A pairs</span>', unsafe_allow_html=True)
    st.markdown('<span class="badge badge-yellow">✓ 100+ responses</span>', unsafe_allow_html=True)
    st.markdown('<span class="badge badge-blue">✓ Emergency ready</span>', unsafe_allow_html=True)

# --- Header ---
col1, col2 = st.columns([4, 1])
with col1:
    st.markdown("<h1>🐾 Jamaica Vet Finder</h1>", unsafe_allow_html=True)
    st.markdown("<p style='color:#6b7280;margin-top:-8px;'>Find veterinary care across Jamaica — chat with Dr. Vetman for help 💙💛</p>", unsafe_allow_html=True)
with col2:
    if st.button("Clear Chat", key="header_clear", use_container_width=True):
        st.session_state.chat_history = []
        st.success("Chat cleared! ✨")
        st.rerun()

# --- Initial greeting ---
if not st.session_state.chat_history:
    greeting = f"🐾 Hi {st.session_state.user_name}! I'm Dr. Vetman. I can help you find veterinary care across Jamaica, answer pet-care questions, and provide emergency guidance. How can I assist you today? 😊"
    st.session_state.chat_history.append(("Dr. Vetman", greeting))

# --- Render chat ---
st.markdown('<div class="chat-wrapper">', unsafe_allow_html=True)
for sender, raw_msg in st.session_state.chat_history:
    text_only = raw_msg if isinstance(raw_msg, str) else str(raw_msg)
    is_user = (sender == st.session_state.user_name)
    avatar = st.session_state.avatar if is_user else BOT_AVATAR
    bubble_cls = "bubble user" if is_user else "bubble bot"

    html_block = f"""
        <div class='chat-row {"user" if is_user else "bot"}'>
            <img class='avatar' src='{avatar}'/>
            <div class='{bubble_cls}'>
                <div class='sender'>{html.escape(sender)}</div>
                <div class='msg-text'>{html.escape(text_only)}</div>
            </div>
        </div>
    """
    st.markdown(html_block, unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# --- Feedback ---
if st.session_state.show_feedback:
    st.markdown("""
        <div class='feedback-modal'>
            <h2 style='text-align:center;margin-bottom:16px;'>⭐ Rate Your Experience</h2>
            <p style='text-align:center;color:var(--muted);margin-bottom:24px;'>
                Thanks for chatting with Dr. Vetman! We'd love your feedback 💙💛
            </p>
        </div>
    """, unsafe_allow_html=True)

    rating = st.slider("How would you rate Dr. Vetman?", 1, 5, 5, key="rating_slider")
    feedback = st.text_area("Any feedback? (optional)", placeholder="Share your thoughts... 🌟", key="feedback_text")

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        if st.button("✨ Submit Feedback", use_container_width=True, key="submit_feedback"):
            st.session_state.star_rating = rating
            st.balloons()
            st.success(f"Thanks for the {rating}⭐ rating! Your feedback helps us improve. 💙💛")
            time.sleep(2)
            st.session_state.show_feedback = False
            st.rerun()

# --- Input area ---
col_left, col_mic, col_send = st.columns([8, 1, 1])

with col_left:
    user_input = st.text_input(
        "Type your question here...",
        key=f"user_input_{len(st.session_state.chat_history)}",
        placeholder="e.g., Find vets in Kingston or say 'hi' 😊",
        label_visibility="collapsed"
    )

with col_mic:
    if WHISPER_OK:
        try:
            voice = mic_recorder(start_prompt="🎤", stop_prompt="⏹️", just_once=True, format="wav", key=f"voice_{len(st.session_state.chat_history)}")
            if isinstance(voice, dict) and voice.get("bytes"):
                transcript = _stt_from_bytes(voice["bytes"])
                if transcript:
                    st.session_state.chat_history.append((st.session_state.user_name, transcript))
                    st.session_state.message_count += 1

                    basic_reply = check_basic_command(transcript)
                    if basic_reply == "FEEDBACK_TRIGGER":
                        reply = "Goodbye! 👋 It was great chatting with you. Take care of your furry friends! 🐾"
                        st.session_state.chat_history.append(("Dr. Vetman", reply))
                        st.session_state.show_feedback = True
                        st.rerun()
                    elif basic_reply:
                        reply = basic_reply
                        audio_path = None
                    else:
                        csv_answer = answer_from_csv(transcript, df, threshold=90)
                        if csv_answer:
                            reply = csv_answer
                            audio_path = None
                        else:
                            reply, audio_path = get_bot_response(transcript)

                    st.session_state.chat_history.append(("Dr. Vetman", reply))
                    st.rerun()
        except:
            st.markdown("🎤")
    else:
        st.markdown("🎤")

with col_send:
    send_clicked = st.button("Send", use_container_width=True)

# --- Handle send ---
if send_clicked and user_input:
    val = user_input.strip()
    if val:
        st.session_state.chat_history.append((st.session_state.user_name, val))
        st.session_state.message_count += 1

        basic_reply = check_basic_command(val)

        if basic_reply == "FEEDBACK_TRIGGER":
            reply = "Goodbye! 👋 It was great chatting with you. Take care of your furry friends! 🐾"
            st.session_state.chat_history.append(("Dr. Vetman", reply))
            st.session_state.show_feedback = True
            st.rerun()
        elif basic_reply:
            reply = basic_reply
            audio_path = None
        else:
            csv_answer = answer_from_csv(val, df, threshold=90)
            if csv_answer:
                reply = csv_answer
                audio_path = None
            else:
                reply, audio_path = get_bot_response(val)

        st.session_state.chat_history.append(("Dr. Vetman", reply))

        if audio_path and AUDIO_BASE_URL:
            try:
                audio_bytes = requests.get(AUDIO_BASE_URL + audio_path, timeout=20).content
                st.audio(audio_bytes, format='audio/mp3')
            except:
                pass

        st.rerun()

# --- Manual Feedback ---
if not st.session_state.show_feedback:
    st.markdown('---')
    col1, col2, col3 = st.columns([1, 1, 1])
    with col2:
        if st.button("⭐ Rate Experience", use_container_width=True, key="manual_feedback"):
            st.session_state.show_feedback = True
            st.rerun()

# --- Footer ---
st.markdown('---')
st.caption("💡 Tip: Try saying 'hi', 'help', 'bye', or ask about vets in your area! 🐾💙💛")

Writing vet_finder_app.py


In [11]:
#100 Basic Responses Cover:

# Greetings (10): hi, hello, hey, good morning, afternoon, evening, night, howdy, yo, greetings
# Farewells (10): bye, goodbye, see you, later, farewell, peace, gotta go, take care, catch you later, see ya
# Thanks (8): thanks, thank you, appreciate it, thx, ty, thanks a lot, many thanks, thank you so much
# About Dr. Vetman (9): how are you, what are you, who are you, what's your name, who made you, what do you do, are you a real vet, are you ai, how old are you
# Help & Capabilities (8): help, what can you do, help me, i need help, can you help me, what services, how can you help, assist me
# Pleasantries (8): how's it going, what's up, how are things, nice to meet you, pleasure to meet you, you're welcome, no problem, all good
# General Questions (9): what time is it, what day is it, where are you, are you busy, are you there, can you hear me, are you real, do you sleep, are you human
# Emotions & Reactions (10): awesome, cool, great, nice, wow, amazing, perfect, excellent, fantastic, wonderful
# Affirmations (10): yes, yeah, yep, sure, okay, ok, no, nope, maybe, perhaps
# Pet-Related (7): i have a dog, i have a cat, i love pets, pets are great, do you like animals, what about birds, what about rabbits
# Compliments (8): you're helpful, you're smart, you're good, you're the best, i like you, you're awesome, good job, well done
# Jamaica Locations (5): where in jamaica, jamaica, kingston, montego bay, spanish town
# Random Fun (5): tell me a joke, are you funny, sing a song, tell me a story, what's your favorite color

In [12]:
# Kill any existing Streamlit process safely
!fuser -k 8501/tcp 2>/dev/null || true

import time

# Give the port time to clear
time.sleep(3)

# --- Run Streamlit App ---
print("🚀 Starting Streamlit...")
!streamlit run vet_finder_app.py --server.headless true --server.port 8501 > /content/streamlit_log.txt 2>&1 &

# Wait for Streamlit to start
time.sleep(8)

# --- Connect ngrok ---
from pyngrok import ngrok

# Kill previous tunnels
ngrok.kill()

# Start new tunnel
public_url = ngrok.connect(8501).public_url

print("✅ Streamlit available at:", public_url)


🚀 Starting Streamlit...
✅ Streamlit available at: https://unvibrated-tubercularly-darwin.ngrok-free.dev
